In [0]:
import requests
import pandas as pd

headers = {
    'accept': 'application/json',
    'x-access-token-ws': 'd1bf4238656c2167c5997e4081b549eedd0a25622a18740d8ad1a8a31ee22b15'
}

# --------------------- FAZENDA --------------------------------------#
url = 'https://api.irancho.com.br/api/fazenda'
response = requests.get(url, headers=headers)
fazenda_list = response.json()
df_fazenda = pd.json_normalize(fazenda_list)
print(f'fazenda {response.status_code}')

In [0]:
display(df_fazenda)